In [19]:
import torch as pt
import torch.nn as nn
from torch.utils.data import random_split
from src.dataset import MolDataset
import numpy as np
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.metrics import f1_score, cohen_kappa_score

In [2]:
device = "cuda" if pt.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
dataset = MolDataset("../data/preprocessed/X.pk", 
                     y_datafile="../data/preprocessed/Y.pk", normal = True)

# Load 

In [4]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.hid = nn.Linear(hidden_size, hidden_size) 
        self.hid2 = nn.Linear(hidden_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.hid(out)
        out = self.relu(out)
        out = self.hid2(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [5]:
n_categories = 3
n_features = 42
learning_rate = 1e-5
n_hidden = 512
# n_layers = config_runtime['layers']


model = NeuralNet(n_features,n_hidden,n_categories).to(device)
model.load_state_dict(pt.load("../model/torch/model.pt"))
model.eval()

NeuralNet(
  (fc1): Linear(in_features=42, out_features=512, bias=True)
  (relu): ReLU()
  (hid): Linear(in_features=512, out_features=512, bias=True)
  (hid2): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=3, bias=True)
)

# Test

In [23]:
ids = [8:2100]

SyntaxError: invalid syntax (2955248105.py, line 1)

In [30]:
X1 = dataset[:][0]
treu_y = dataset[:][1]
X = pt.tensor(X1, device=device)

logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)

In [8]:
# treu_y' pred_probab

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [31]:
pred_cats = np.array(pred_probab.argmax(2).cpu()[0])
true_cat = treu_y.argmax(1)

In [32]:
print(pred_cats, true_cat)

[1 0 2 ... 1 0 2] [0 0 0 ... 2 2 2]


In [33]:
score = f1_score(pred_cats, true_cat, average=None)
score2 = cohen_kappa_score(pred_cats, true_cat)

In [36]:
print(score, 1/score2)

[0.07987241 0.09279098 0.658779  ] 34.024649270853324


# Real predictions


In [ ]:
X_test = pd.read_pickle("../data/preprocessed/test.pk")
scaler = joblib.load("../model/torch/scaler.save") 

In [ ]:
X_test_mat = np.array(X_test.loc[:, X_test.columns != 'Id'].values,
                 dtype=np.float32)
X_test_mat = scaler.fit_transform(X_test_mat)
X_id = list(X_test.loc[:,'Id'])

In [ ]:
X = pt.tensor(X_test_mat[None, :,:], device=device)
# # X = torch.unsqueeze(X, dim=0)

logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)

In [ ]:
X

In [ ]:
pred_probab

In [ ]:
vals = np.array(pred_probab.argmax(2).cpu())[0]

In [ ]:
report = pd.DataFrame({'Id': X_id, 'pred': vals})
report.to_csv("submission.csv", index=False)